# Векторный поиск похожих сортов пива

## Описание
Эта секция создает FAISS индекс для векторного поиска похожих сортов пива по текстовым запросам пользователя.

## Как это работает:
1. Загружаем собранные данные о пиве из JSON
2. Создаем текстовое представление каждого пива (название + описание + теги)
3. Генерируем векторные эмбеддинги с помощью sentence-transformers
4. Строим FAISS индекс для быстрого поиска
5. По запросу пользователя находим топ-10 наиболее похожих сортов

## ⚠️ Предварительные требования

### Установка библиотек
Перед запуском убедитесь, что установлены все необходимые библиотеки:

```bash
pip install faiss-cpu sentence-transformers numpy
```

Или используйте файл requirements.txt:
```bash
pip install -r requirements.txt
```

### Данные
Убедитесь, что файл `beer_data_full.json` существует в текущей директории.
Если его нет, сначала запустите сбор данных в `parser.ipynb`.

## Использование:
```python
# Поиск пива по запросу
results = search_similar_beers("Хочу освежающее светлое пиво с цитрусовыми нотками", top_k=10)
```


In [4]:
# ШАГИ 1-2: Импорт библиотек и загрузка данных

import json
import pickle
from typing import List, Dict, Tuple

print("📦 Загрузка библиотек для векторного поиска...")

# Проверяем наличие необходимых библиотек
try:
    import faiss
    import numpy as np
    from sentence_transformers import SentenceTransformer
    print("✅ Все необходимые библиотеки установлены")
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")
    print("\n⚠️  Установите необходимые библиотеки:")
    print("pip install faiss-cpu sentence-transformers numpy")
    raise

# Загружаем модель для создания эмбеддингов
# Используем многоязычную модель, которая хорошо работает с русским и английским
print("🤖 Загрузка модели sentence-transformers...")
print("   (Первая загрузка может занять несколько минут)")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(f"✅ Модель загружена. Размерность эмбеддингов: {embedding_model.get_sentence_embedding_dimension()}")

# Загружаем данные о пиве
def load_beer_data(json_file: str = "beer_data_full.json") -> List[Dict]:
    """Загрузка данных о пиве из JSON файла"""
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        # Фильтруем только успешные записи (без ошибок)
        beers = [beer for beer in data if 'error' not in beer and beer.get('description')]
        print(f"📊 Загружено {len(beers)} сортов пива (из {len(data)} всего)")
        return beers
    except FileNotFoundError:
        print(f"❌ Файл {json_file} не найден")
        print("   Сначала запустите сбор данных в parser.ipynb")
        return []
    except Exception as e:
        print(f"❌ Ошибка при загрузке данных: {e}")
        return []

# Загружаем данные
beer_data = load_beer_data()

if not beer_data:
    print("⚠️  Данные не загружены. Сначала запустите сбор данных!")
else:
    print(f"✅ Данные готовы к обработке: {len(beer_data)} сортов пива")


📦 Загрузка библиотек для векторного поиска...


/Users/dlyapin/.pyenv/versions/3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Все необходимые библиотеки установлены
🤖 Загрузка модели sentence-transformers...
   (Первая загрузка может занять несколько минут)
✅ Модель загружена. Размерность эмбеддингов: 384
📊 Загружено 531 сортов пива (из 532 всего)
✅ Данные готовы к обработке: 531 сортов пива


In [5]:
# ШАГ 3: Создание текстовых представлений пива для эмбеддингов

def create_beer_text(beer: Dict) -> str:
    """
    Создает текстовое представление пива для генерации эмбеддингов.
    Объединяет название, описание, страну, тип, стиль и теги.
    """
    parts = []
    
    # Название
    if beer.get('name'):
        parts.append(f"Название: {beer['name']}")
    
    # Описание
    if beer.get('description'):
        parts.append(f"Описание: {beer['description']}")
    
    # Страна
    if beer.get('country'):
        parts.append(f"Страна: {beer['country']}")
    
    # Тип пива
    if beer.get('beer_type'):
        parts.append(f"Тип: {beer['beer_type']}")
    
    # Стиль
    if beer.get('style'):
        parts.append(f"Стиль: {beer['style']}")
    
    # Крепость
    if beer.get('alcohol_percentage'):
        parts.append(f"Алкоголь: {beer['alcohol_percentage']}")
    
    # Горечь
    if beer.get('bitterness'):
        parts.append(f"Горечь: {beer['bitterness']}")
    
    # Вкусовые теги
    if beer.get('taste_tags'):
        tags = beer['taste_tags']
        if isinstance(tags, list) and tags:
            parts.append(f"Вкус: {', '.join(tags)}")
    
    return " | ".join(parts)

# Создаем текстовые представления для всех сортов пива
print("📝 Создание текстовых представлений...")
beer_texts = []
valid_beers = []

for beer in beer_data:
    text = create_beer_text(beer)
    if text:  # Проверяем, что текст не пустой
        beer_texts.append(text)
        valid_beers.append(beer)

print(f"✅ Создано {len(beer_texts)} текстовых представлений")

# Пример одного текстового представления
if beer_texts:
    print(f"\n📄 Пример текстового представления:")
    print(f"{beer_texts[0][:300]}...")


📝 Создание текстовых представлений...
✅ Создано 531 текстовых представлений

📄 Пример текстового представления:
Название: Corona Extra | Описание: Corona Extra - это легкое и освежающее светлое пиво, которое известно своим мягким вкусом и легкой горечью. Оно обладает светло-золотистым цветом и часто подается с долькой лайма, что придает ему дополнительную свежесть. Corona Extra имеет легкий аромат солода и ку...


In [6]:
# ШАГ 4: Генерация эмбеддингов и создание FAISS индекса

print("🔄 Генерация эмбеддингов для всех сортов пива...")
print("⏱️  Это может занять несколько минут...")

# Генерируем эмбеддинги пакетами для ускорения
batch_size = 32
embeddings = embedding_model.encode(
    beer_texts, 
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"✅ Созданы эмбеддинги размерностью {embeddings.shape}")

# Нормализуем эмбеддинги для косинусного сходства
faiss.normalize_L2(embeddings)

# Создаем FAISS индекс
embedding_dim = embeddings.shape[1]
print(f"\n🔨 Создание FAISS индекса размерности {embedding_dim}...")

# Используем IndexFlatIP для косинусного сходства (после нормализации)
index = faiss.IndexFlatIP(embedding_dim)

# Добавляем эмбеддинги в индекс
index.add(embeddings)

print(f"✅ FAISS индекс создан. Проиндексировано {index.ntotal} сортов пива")

# Сохраняем индекс и данные для последующего использования
print("\n💾 Сохранение индекса и данных...")

# Сохраняем FAISS индекс
faiss.write_index(index, "beer_faiss.index")

# Сохраняем данные о пиве
with open("beer_data_indexed.pkl", "wb") as f:
    pickle.dump(valid_beers, f)

print("✅ Индекс и данные сохранены:")
print("   - beer_faiss.index (FAISS индекс)")
print("   - beer_data_indexed.pkl (данные о пиве)")


🔄 Генерация эмбеддингов для всех сортов пива...
⏱️  Это может занять несколько минут...


Batches: 100%|██████████| 17/17 [00:02<00:00,  6.21it/s]

✅ Созданы эмбеддинги размерностью (531, 384)

🔨 Создание FAISS индекса размерности 384...
✅ FAISS индекс создан. Проиндексировано 531 сортов пива

💾 Сохранение индекса и данных...
✅ Индекс и данные сохранены:
   - beer_faiss.index (FAISS индекс)
   - beer_data_indexed.pkl (данные о пиве)


In [7]:
# ШАГ 5: Функция поиска похожих сортов пива

class BeerRecommender:
    """Класс для поиска похожих сортов пива по текстовому запросу"""
    
    def __init__(self, index_path: str = "beer_faiss.index", data_path: str = "beer_data_indexed.pkl"):
        """Инициализация рекомендательной системы"""
        self.embedding_model = embedding_model  # Используем уже загруженную модель
        
        # Загружаем FAISS индекс
        print("📂 Загрузка FAISS индекса...")
        self.index = faiss.read_index(index_path)
        
        # Загружаем данные о пиве
        print("📂 Загрузка данных о пиве...")
        with open(data_path, "rb") as f:
            self.beers = pickle.load(f)
        
        print(f"✅ Загружено {len(self.beers)} сортов пива")
    
    def search(self, query: str, top_k: int = 10) -> List[Tuple[Dict, float]]:
        """
        Поиск похожих сортов пива по текстовому запросу
        
        Args:
            query: Текстовый запрос пользователя
            top_k: Количество результатов для возврата
        
        Returns:
            Список кортежей (beer_data, similarity_score)
        """
        # Генерируем эмбеддинг для запроса
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)
        
        # Нормализуем для косинусного сходства
        faiss.normalize_L2(query_embedding)
        
        # Ищем ближайшие векторы
        similarities, indices = self.index.search(query_embedding, top_k)
        
        # Формируем результаты
        results = []
        for idx, similarity in zip(indices[0], similarities[0]):
            beer = self.beers[idx]
            results.append((beer, float(similarity)))
        
        return results
    
    def print_results(self, results: List[Tuple[Dict, float]]):
        """Красиво выводит результаты поиска"""
        print("\n" + "="*80)
        print(f"🍺 НАЙДЕНО {len(results)} ПОХОЖИХ СОРТОВ ПИВА")
        print("="*80)
        
        for i, (beer, score) in enumerate(results, 1):
            print(f"\n{i}. 🍺 {beer.get('name', 'Неизвестно')}")
            print(f"   📊 Схожесть: {score:.2%}")
            print(f"   🌍 Страна: {beer.get('country', 'н/д')}")
            print(f"   🎨 Стиль: {beer.get('style', 'н/д')}")
            print(f"   🍺 Тип: {beer.get('beer_type', 'н/д')}")
            print(f"   🔥 Алкоголь: {beer.get('alcohol_percentage', 'н/д')}")
            print(f"   😋 Горечь: {beer.get('bitterness', 'н/д')}")
            
            tags = beer.get('taste_tags', [])
            if tags and isinstance(tags, list):
                print(f"   🏷️  Теги: {', '.join(tags)}")
            
            description = beer.get('description', '')
            if description:
                # Обрезаем описание до 200 символов
                desc_short = description[:200] + "..." if len(description) > 200 else description
                print(f"   📝 {desc_short}")

# Создаем экземпляр рекомендательной системы
print("🚀 Инициализация рекомендательной системы...")
recommender = BeerRecommender()
print("✅ Система готова к работе!")


🚀 Инициализация рекомендательной системы...
📂 Загрузка FAISS индекса...
📂 Загрузка данных о пиве...
✅ Загружено 531 сортов пива
✅ Система готова к работе!


In [8]:
# ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ: Поиск пива по различным запросам

# Пример 1: Поиск освежающего светлого пива
print("🔍 Пример 1: Поиск освежающего светлого пива с цитрусовыми нотками")
query1 = "Хочу освежающее светлое пиво с цитрусовыми нотками"
results1 = recommender.search(query1, top_k=5)
recommender.print_results(results1)

# Пример 2: Поиск темного пива
print("\n\n🔍 Пример 2: Поиск крепкого темного пива с карамельным вкусом")
query2 = "Ищу крепкое темное пиво с карамельными и шоколадными нотками"
results2 = recommender.search(query2, top_k=5)
recommender.print_results(results2)


# Пример 3: Поиск IPA
print("\n\n🔍 Пример 3: Поиск горького хмельного IPA")
query3 = "Хочу горькое хмельное пиво в стиле IPA с высоким содержанием алкоголя"
results3 = recommender.search(query3, top_k=5)
recommender.print_results(results3)


# Пример 4: Поиск по стране и стилю
print("\n\n🔍 Пример 4: Поиск бельгийского пшеничного пива")
query4 = "Бельгийское пшеничное пиво с фруктовыми нотами"
results4 = recommender.search(query4, top_k=5)
recommender.print_results(results4)

# Пример 5: Простой запрос
print("\n\n🔍 Пример 5: Поиск легкого пива для жаркого дня")
query5 = "Легкое пиво для жаркого летнего дня"
results5 = recommender.search(query5, top_k=5)
recommender.print_results(results5)




🔍 Пример 1: Поиск освежающего светлого пива с цитрусовыми нотками

🍺 НАЙДЕНО 5 ПОХОЖИХ СОРТОВ ПИВА

1. 🍺 Балтика Лимон
   📊 Схожесть: 67.17%
   🌍 Страна: Россия
   🎨 Стиль: ароматизированное пиво
   🍺 Тип: светлое
   🔥 Алкоголь: 2.5
   😋 Горечь: низкая
   🏷️  Теги: освежающее, цитрусовое, легкое
   📝 Балтика Лимон - это освежающее пиво с добавлением натурального лимонного сока, которое идеально подходит для жарких летних дней. Оно обладает легким и освежающим вкусом с яркими цитрусовыми нотками. Э...

2. 🍺 Балтика Пшеничное Лайм
   📊 Схожесть: 66.79%
   🌍 Страна: Россия
   🎨 Стиль: пшеничное
   🍺 Тип: светлое
   🔥 Алкоголь: 4.0
   😋 Горечь: низкая
   🏷️  Теги: лайм, цитрусовый, освежающий, фруктовый, легкий
   📝 Балтика Пшеничное Лайм - это освежающее пиво с добавлением натурального сока лайма, которое сочетает в себе легкость и фруктовые нотки. Оно обладает мягким вкусом с цитрусовыми оттенками и легкой слад...

3. 🍺 Афанасий Светлое
   📊 Схожесть: 65.65%
   🌍 Страна: Россия
   🎨 Стил

In [10]:
# ФУНКЦИЯ ДЛЯ БЫСТРОГО ПОИСКА: Удобная обертка

def search_similar_beers(query: str, top_k: int = 10, show_full_description: bool = False) -> List[Dict]:
    """
    Удобная функция для поиска похожих сортов пива
    
    Args:
        query: Текстовый запрос пользователя
        top_k: Количество результатов (по умолчанию 10)
        show_full_description: Показывать полное описание
    
    Returns:
        Список словарей с информацией о найденных сортах пива
    """
    results = recommender.search(query, top_k=top_k)
    
    # Формируем список для возврата
    beers_list = []
    
    print("\n" + "="*80)
    print(f"🔍 ЗАПРОС: {query}")
    print(f"🍺 НАЙДЕНО: {len(results)} похожих сортов")
    print("="*80)
    
    for i, (beer, score) in enumerate(results, 1):
        print(f"\n{i}. 🍺 {beer.get('name', 'Неизвестно')} (схожесть: {score:.1%})")
        print(f"   🌍 {beer.get('country', 'н/д')} | "
              f"🎨 {beer.get('style', 'н/д')} | "
              f"🍺 {beer.get('beer_type', 'н/д')}")
        print(f"   🔥 {beer.get('alcohol_percentage', 'н/д')} | "
              f"😋 {beer.get('bitterness', 'н/д')}")
        
        tags = beer.get('taste_tags', [])
        if tags and isinstance(tags, list):
            print(f"   🏷️  {', '.join(tags)}")
        
        description = beer.get('description', '')
        if description:
            if show_full_description:
                print(f"   📝 {description}")
            else:
                desc_short = description[:150] + "..." if len(description) > 150 else description
                print(f"   📝 {desc_short}")
        
        # Добавляем в список для возврата
        beer_with_score = beer.copy()
        beer_with_score['similarity_score'] = score
        beers_list.append(beer_with_score)
    
    return beers_list

# Пример использования функции
print("✅ Функция search_similar_beers() готова к использованию!")


✅ Функция search_similar_beers() готова к использованию!


In [16]:
# Пример
results = search_similar_beers("Подбери мне темное пиво на вечер как гинесс но дешевле", top_k=5)


🔍 ЗАПРОС: Подбери мне темное пиво на вечер как гинесс но дешевле
🍺 НАЙДЕНО: 5 похожих сортов

1. 🍺 Аливария Темное (схожесть: 64.3%)
   🌍 Беларусь | 🎨 лагер | 🍺 темное
   🔥 4.5 | 😋 низкая
   🏷️  карамель, шоколад, мягкое, бархатистое, насыщенное
   📝 Аливария Темное - это насыщенное темное пиво с глубоким ароматом и богатым вкусом. Оно обладает карамельными и шоколадными нотками, которые прекрасно ...

2. 🍺 Балтика Кулер (схожесть: 62.7%)
   🌍 Россия | 🎨 лагер | 🍺 светлое
   🔥 4.7 | 😋 низкая
   🏷️  освежающее, легкое, хмельное, солодовое
   📝 Балтика Кулер - это освежающее светлое пиво, которое идеально подходит для жарких летних дней. Оно обладает легким вкусом с нотками хмеля и солода, чт...

3. 🍺 Балтика 1 Легкое (схожесть: 62.4%)
   🌍 Россия | 🎨 лагер | 🍺 светлое
   🔥 4.0 | 😋 низкая
   🏷️  легкое, освежающее, сладость, светлое
   📝 Балтика 1 Легкое - это освежающее светлое пиво с легким и мягким вкусом. Оно обладает приятной сладостью и легкой горечью, что делает его идеальным дл.

In [1]:
from beer_search import (
    search_similar_beers,
    search_beers_by_filter,
    BeerRecommender,
    get_recommender
)

results = search_similar_beers("Подбери мне темное пиво на вечер как гинесс но дешевле", top_k=5)

/Users/dlyapin/.pyenv/versions/3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Загрузка модели paraphrase-multilingual-MiniLM-L12-v2...
📂 Загрузка FAISS индекса из beer_faiss.index...
📂 Загрузка данных о пиве из beer_data_indexed.pkl...
✅ Загружено 531 сортов пива
✅ Размерность индекса: 384

🔍 ЗАПРОС: Подбери мне темное пиво на вечер как гинесс но дешевле
🍺 НАЙДЕНО: 5 похожих сортов

1. 🍺 Аливария Темное (схожесть: 64.3%)
   🌍 Беларусь | 🎨 лагер | 🍺 темное
   🔥 4.5 | 😋 низкая
   🏷️  карамель, шоколад, мягкое, бархатистое, насыщенное
   📝 Аливария Темное - это насыщенное темное пиво с глубоким ароматом и богатым вкусом. Оно обладает карамельными и шоколадными нотками, которые прекрасно ...

2. 🍺 Балтика Кулер (схожесть: 62.7%)
   🌍 Россия | 🎨 лагер | 🍺 светлое
   🔥 4.7 | 😋 низкая
   🏷️  освежающее, легкое, хмельное, солодовое
   📝 Балтика Кулер - это освежающее светлое пиво, которое идеально подходит для жарких летних дней. Оно обладает легким вкусом с нотками хмеля и солода, чт...

3. 🍺 Балтика 1 Легкое (схожесть: 62.4%)
   🌍 Россия | 🎨 лагер | 🍺 светлое
   🔥 4

In [3]:
results

[{'name': 'Аливария Темное',
  'description': 'Аливария Темное - это насыщенное темное пиво с глубоким ароматом и богатым вкусом. Оно обладает карамельными и шоколадными нотками, которые прекрасно сочетаются с легкой горечью. Это пиво идеально подходит для тех, кто ценит сложные и многогранные вкусы. Оно обладает мягким и бархатистым послевкусием, которое оставляет приятное впечатление. Аливария Темное - отличный выбор для уютных вечеров и дружеских встреч.',
  'alcohol_percentage': 4.5,
  'bitterness': 'низкая',
  'country': 'Беларусь',
  'taste_tags': ['карамель', 'шоколад', 'мягкое', 'бархатистое', 'насыщенное'],
  'beer_type': 'темное',
  'style': 'лагер',
  'similarity_score': 0.6434328556060791},
 {'name': 'Балтика Кулер',
  'description': 'Балтика Кулер - это освежающее светлое пиво, которое идеально подходит для жарких летних дней. Оно обладает легким вкусом с нотками хмеля и солода, что делает его приятным для питья. Пиво отличается своей легкостью и низким содержанием алкогол